# Strategy 1 & 2: HYG Trading Strategies

Implementation of Strategy 1 and Strategy 2 based on BMOM (Blended Momentum) signals.

## Strategy 1 Logic:

```
thresh = -0.03

if (((SPY.BMOM(i-1) > thresh) && (SPY.BMOM(i) < thresh) && (HYG.BMOM(i) < thresh)) ||
    ((HYG.BMOM(i-1) > thresh) && (HYG.BMOM(i) < thresh) && (SPY.BMOM(i) < thresh))) &&
    (HYG.mom240(i) < 0)
    HYG_Strat1_Wgt = 0;
else
    if HYG.BMOM(i) > 0
        HYG_Strat1_Wgt = 1;
    else
        HYG_Strat1_Wgt(i) = HYG_Strat1_Wgt(i-1);
    end
end
```

## Strategy 2 Logic:

```
drawdiffthresh = 0.10

If (Strat1_drawdown - HYG_drawdown) > drawdiffthresh:
    HYG_Strat2_Wgt = 1  (override to long)
Else:
    Use same rules as Strat1 (signal = 0 or 1)
```

**Free Parameters:**
- w1, w2, w3, w4 (BMOM weights)
- thresh (threshold for exit)
- drawdiffthresh (drawdown differential threshold)
- mom240 length (momentum period for HYG in Strat1)

In [4]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Imports loaded")

Imports loaded


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Portfolio Constraints
CONSTRAINTS = {
    'Cash': {'min': 0.05, 'max': 0.10},
    'H5A4': {'min': 0.00, 'max': 0.12},  # HY capped at 12%
    'C5A0': {'min': 0.00, 'max': 0.95},
    'G5O2': {'min': 0.00, 'max': 0.95}
}

# Strategy 1: Signal Generation Thresholds (per asset)
SIGNAL_THRESHOLDS = {
    'H5A4': -0.03,   # Exit threshold vs SPY
    'C5A0': -0.013,  # Exit threshold vs LQD
    'G5O2': -0.01    # Exit threshold vs TLT
}

# Defensive Allocation (when no signals active)
DEFENSIVE_ALLOCATION = {'Cash': 0.10, 'H5A4': 0.00, 'C5A0': 0.00, 'G5O2': 0.90}

print("Configuration loaded")
print(f"\nSignal Thresholds:")
for asset, thresh in SIGNAL_THRESHOLDS.items():
    print(f"  {asset}: {thresh}")
print(f"\nDefensive Allocation: {DEFENSIVE_ALLOCATION}")
print(f"\nNote: Strategy 2 DD override thresholds are set in individual asset analysis files")

Configuration loaded

Signal Thresholds:
  H5A4: -0.03
  C5A0: -0.013
  G5O2: -0.01

Defensive Allocation: {'Cash': 0.1, 'H5A4': 0.0, 'C5A0': 0.0, 'G5O2': 0.9}

Note: Strategy 2 DD override thresholds are set in individual asset analysis files


In [6]:
# =============================================================================
# DATA LOADING
# =============================================================================

# Load ETF data from individual CSV files
spy_data = pd.read_csv(r'Data Files/SPY 1.csv', 
                       parse_dates=['Date'], index_col='Date')
hyg_data = pd.read_csv(r'Data Files/HYG.csv', 
                       skiprows=15, parse_dates=['Date'], index_col='Date', usecols=['Date', 'Price'])
lqd_data = pd.read_csv(r'Data Files/LQD.csv', 
                       skiprows=15, parse_dates=['Date'], index_col='Date', usecols=['Date', 'Price'])
tlt_data = pd.read_csv(r'Data Files/TLT.csv', 
                       skiprows=15, parse_dates=['Date'], index_col='Date', usecols=['Date', 'Price'])

# Load asset-specific data from CSV files
h5a4_data = pd.read_csv(r'Data files/H5A4.csv', 
                        parse_dates=['Date'], index_col='Date')
c5a0_data = pd.read_csv(r'Data files/C5A0.csv', 
                        parse_dates=['Date'], index_col='Date')
g5o2_data = pd.read_csv(r'Data files/G5O2.csv', 
                        parse_dates=['Date'], index_col='Date')

# Calculate BMOM for each asset (weighted momentum)
def calc_bmom(prices):
    return (0.15 * prices.pct_change(20) + 0.35 * prices.pct_change(60) + 
            0.35 * prices.pct_change(120) + 0.15 * prices.pct_change(240))

# Calculate benchmark BMOMs from ETF data
SPY_BMOM = calc_bmom(spy_data['SPY_adj_close'])
HYG_BMOM = calc_bmom(hyg_data['Price'])
LQD_BMOM = calc_bmom(lqd_data['Price'])
TLT_BMOM = calc_bmom(tlt_data['Price'])

# Calculate asset BMOMs and Mom240
H5A4_BMOM = calc_bmom(h5a4_data['H5A4'])
C5A0_BMOM = calc_bmom(c5a0_data['C5A0'])
G5O2_BMOM = calc_bmom(g5o2_data['G5O2'])

H5A4_Mom240 = h5a4_data['H5A4'].pct_change(240)
C5A0_Mom240 = c5a0_data['C5A0'].pct_change(240)
G5O2_Mom240 = g5o2_data['G5O2'].pct_change(240)

# Consolidate into single DataFrame
data = pd.DataFrame()

# Benchmark BMOMs
data['SPY_BMOM'] = SPY_BMOM
data['TLT_BMOM'] = TLT_BMOM
data['LQD_BMOM'] = LQD_BMOM
data['HYG_BMOM'] = HYG_BMOM

# Asset BMOMs
data['H5A4_BMOM'] = H5A4_BMOM
data['C5A0_BMOM'] = C5A0_BMOM
data['G5O2_BMOM'] = G5O2_BMOM

# Asset Prices
data['H5A4_Price'] = h5a4_data['H5A4']
data['C5A0_Price'] = c5a0_data['C5A0']
data['G5O2_Price'] = g5o2_data['G5O2']

# Mom240 (240-day momentum for exit filter)
data['H5A4_Mom240'] = H5A4_Mom240
data['C5A0_Mom240'] = C5A0_Mom240
data['G5O2_Mom240'] = G5O2_Mom240

# Sort and clean
data = data.sort_index()
data.index = pd.to_datetime(data.index)
data.index.name = 'Date'

# Filter to first common date (no NaN in core columns)
core_cols = ['SPY_BMOM', 'TLT_BMOM', 'LQD_BMOM', 'HYG_BMOM', 
             'H5A4_BMOM', 'C5A0_BMOM', 'G5O2_BMOM',
             'H5A4_Price', 'C5A0_Price', 'G5O2_Price', 
             'H5A4_Mom240', 'C5A0_Mom240', 'G5O2_Mom240']
first_date = data[core_cols].dropna().index.min()
data = data.loc[first_date:].copy()

# Calculate daily returns
data['H5A4_Return'] = data['H5A4_Price'].pct_change()
data['C5A0_Return'] = data['C5A0_Price'].pct_change()
data['G5O2_Return'] = data['G5O2_Price'].pct_change()

print(f"Data loaded successfully")
print(f"  Shape: {data.shape}")
print(f"  Date range: {data.index.min().date()} to {data.index.max().date()}")
print(f"  Trading days: {len(data):,}")

Data loaded successfully
  Shape: (4439, 16)
  Date range: 2008-03-25 to 2025-11-11
  Trading days: 4,439
